In [21]:
import hnswlib
import numpy as np
import numpy as np
import h5py
import os
import requests
import tempfile
import time
import struct
import itertools

from core_utils import read_float_binary_file,read_i8_binary_file,read_u8_binary_file,write_float_binary_file,read_int32_binary_file,compute_recall,get_bin_metadata,omp_sum_of_squares



In [22]:
query_file = '/home/rakri/wiki_rnd1m/wikipedia_query.bin'
gt_file = '/home/rakri/wikipedia_gt_unfilt.bin'
base_file = '/home/rakri/wikipedia/mem_r64_l100.data'
dataset_name='wiki_large'

#gt_file = '/home/rakri/wiki_rnd1m_gt100.bin'
#base_file = '/home/rakri/wiki_rnd1m_data.bin'
#dataset_name='wiki_rnd1m'


[nqgt, nkgt, gt] = read_int32_binary_file(gt_file)
[nq, ndq, queries] = read_float_binary_file(query_file)
[nb, nd] = get_bin_metadata(base_file)

print ("queries,gt, and base metadata loaded")

#names and parameters
Mb = 16
efb = 200
index_name = '/home/rakri/hnsw_'+dataset_name+'M='+str(Mb)+'_efC='+str(efb) 

# for hnsw, it is only ef_search
search_params = [(10), (20), (30), (40), (50), (60), (70), (80), (90), (100), (150), (200), (250), (300)]


# Create a NumPy array.
A = np.array([[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3], [4, 5, 6, 4, 5, 6, 4, 5, 6, 4, 5, 6], [7, 8, 9, 7, 8, 9, 7, 8, 9, 7, 8, 9]])


queries,gt, and base metadata loaded


In [19]:
[nb, nd, dataset] = read_float_binary_file(base_file)

In [23]:
#arr = np.array([[1, 2, 3], [4, 5, 6], [1, 2, 3]])
#row_sums = np.sum(dataset, axis=1)
#row_sums = np.sum(dataset ** 2, axis=1)
row_sums = omp_sum_of_squares(dataset)
print(row_sums.shape)
# Create an array
#arr = np.array([1, 2, 3, 1, 4, 5, 2, 5])

# Get the unique values
unique_elements = np.unique(row_sums)

# Print the unique values
print(unique_elements.shape)

In [ ]:
# Create an HNSW index
index = hnswlib.Index(space='l2', dim=nd)

index.init_index(max_elements=nb, M = Mb, ef_construction = efb, random_seed = 100)

print ("index initialized")
if os.path.isfile(index_name):
    print("Index exists, so loading")
    index = hnswlib.Index(space='l2', dim=nd)
    index.load_index(index_name, max_elements = nb)
else:
    #index.set_num_threads()
    start = time.time()
    [nb, nd, dataset] = read_float_binary_file(base_file)
    index.add_items(dataset)
    end = time.time()
    print("Index built in", end-start, " seconds.")
    index.save_index(index_name)


Index exists, so loading


In [ ]:
# %%
# Search for the nearest neighbor of a query vector

index.set_num_threads(1)
nk=10
print("ef\tRecall\t\ttime")
for param in search_params:
    (ef_search) = param
    index.set_ef(ef_search)
    start = time.time()
    neighbors, distances = index.knn_query(queries, k=nk)
    end = time.time()
    recall = compute_recall(neighbors, gt[:,:nk])
    print(ef_search,"\t",recall,"\t",1000000*(end-start)/nq)
# Print the nearest neighbor
#print(neighbors)
#print(distances)




ef	Recall		time


10 	 0.68444 	 61.86943054199219
20 	 0.73324 	 114.05119895935059
30 	 0.82468 	 162.32924461364746
40 	 0.88636 	 207.39431381225586
50 	 0.9136 	 247.88260459899902
60 	 0.93184 	 286.47656440734863
70 	 0.95938 	 324.2431640625
80 	 0.98924 	 362.2021198272705
90 	 0.99904 	 401.24807357788086
100 	 0.99988 	 439.23306465148926
